# This notebook is for cleaning the BOLD data associated with the HCP movie-watching subjects.


## 1. BOLD processing for encoding models
Section 1 processes BOLD into format required for encoding models, using only lh and rh.  
If you want to look at subcortical voxels too, you'll need to add that in when generating Y_test and Y_train.  

Ending framework is as follows:  
training data (TR, vertices)  
testing data (repeats, TR, vertices)  

import stuff

In [1]:
import numpy as np
import neuropythy as ny
import json

create bold dictionary

In [2]:
bolddictionary = dict()
movies = ['MOVIE1_7T_AP', 'MOVIE2_7T_PA', 'MOVIE3_7T_PA', 'MOVIE4_7T_AP']

for i in movies:
    filename = f'/home/jovyan/shared/HCP/115825/MNINonLinear/Results/tfMRI_{i}/tfMRI_{i}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
    cii = ny.load(filename)
    (lh_bold, rh_bold, subcortex_bold) = ny.hcp.cifti_split(cii)
    
    bolddictionary[f'{i}'] = {'lh_bold':lh_bold, 'rh_bold':rh_bold, 'subcortex_bold':subcortex_bold}

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


select only TRs that correspond to train and test

In [ ]:
json_filepath = '/home/jovyan/visual-feature-decoding/extract_features/feature.json'

with open(json_filepath) as f:
    data = json.load(f)

# load in json data
for stimuli, stimuli_data in data.items():
        movies = stimuli_data['movies']
        savepath = stimuli_data['savepath']
        TRs = stimuli_data['TRs']
        
        y_train_lh = []
        y_train_rh = []
        y_train_subcortex = []
        y_test_lh = []
        y_test_rh = []
        y_test_subcortex = []
        
        for movie, run_TRs in TRs.items():
            movie_feature = movie[3:-7]
            for keys, values in bolddictionary.items():
                for keyx, region in values.items():
                    movie_bold = keys[:-6]
                    if movie_feature == movie_bold:
                        for key, run in run_TRs.items():
                            if key.startswith('train'):
                                if keyx == 'lh_bold':
                                    y_train_lh.append(region[run[0]:run[1],:])
                                if keyx == 'rh_bold':
                                    y_train_rh.append(region[run[0]:run[1],:])
                                if keyx == 'subcortex_bold':
                                    y_train_subcortex.append(region[run[0]:run[1],:])
                            if key.startswith('test'):
                                if keyx == 'lh_bold':
                                    y_test_lh.append(region[run[0]:run[1],:])
                                if keyx == 'rh_bold':
                                    y_test_rh.append(region[run[0]:run[1],:])
                                if keyx == 'subcortex_bold':
                                    y_test_subcortex.append(region[run[0]:run[1],:])


Create Y_train and Y_test dictionaries, and flatten into numpy array

In [4]:
Y_train_regions = dict()
trainlist = {'y_train_rh': y_train_rh, 'y_train_lh': y_train_lh, 'y_train_subcortex': y_train_subcortex}

for name, train in trainlist.items():
    intermediate = np.concatenate(train, axis=0)
    Y_train_regions[f'{name}'] = intermediate
    
Y_test_regions = dict()
testlist = {'y_test_rh': y_test_rh, 'y_test_lh': y_test_lh, 'y_test_subcortex': y_test_subcortex}

for name, test in testlist.items():
    intermediate = np.stack(test)
    Y_test_regions[f'{name}'] = intermediate
    
# flatten to numpy arrays
Y_test = np.concatenate((Y_test_regions['y_test_rh'], Y_test_regions['y_test_lh']), axis=2)
Y_train = np.concatenate((Y_train_regions['y_train_rh'], Y_train_regions['y_train_lh']), axis=1)

info for saving with file

In [18]:
info = 'Y_test is the concatenation of rh and lh, rh first. this bold data is cleaned for encoding models!'

save out bold!

In [ ]:
np.savez('/home/jovyan/workingdirectory/encodingmodel_BOLD.npz', Y_train=Y_train, Y_test=Y_test, info=info)
Y_train.shape, Y_test.shape, info

# 2. BOLD processing for classifier
This section is for processing the BOLD into a pandas dataframe with columns subject, TR, BOLD, hemisphere, and features (if added)

import stuff

In [16]:
import numpy as np
import neuropythy as ny
import json
import nibabel as nib
import pandas as pd

load in the ROIs for the subject. this is using HCP's surface, FSLR? 

In [17]:
ROI = nib.load('/home/jovyan/shared/HCP/115825/MNINonLinear/fsaverage_LR59k/115825.aparc.59k_fs_LR.dlabel.nii')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [18]:
(lh_roi, rh_roi, subcortex_roi) = ny.hcp.cifti_split(ROI)


I went through visualizing each ROI to identify which ones are in visual cortex. the ```visual_ROIs``` list is the list of ROIs that make up visual cortex. 

In [19]:
visual_ROIs = [5, 8, 11, 13, 21]

Now let's merge them to get a huge visual cortex mask

In [21]:
hemi = {'lh':lh_roi[0], 'rh':lh_roi[0]}

visualcortexmask = dict()
for h, roi in hemi.items():
    for ROI in visual_ROIs:
        result = ((roi == 5) | (roi == 8) | (roi == 11) | (roi == 13) | (roi == 21))
        visualcortexmask[f"{h}"] = result
        
    

Let's visualize on the surface!

In [22]:
sub = ny.hcp_subject('/home/jovyan/shared/HCP/115825')
ny.cortex_plot(sub.hemis['lh_LR59k'], color='r', mask=(visualcortexmask['lh']))

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…

In [23]:
sub = ny.hcp_subject('/home/jovyan/shared/HCP/115825')
ny.cortex_plot(sub.hemis['rh_LR59k'], color='r', mask=(visualcortexmask['lh']))

Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…

Great!  
Now let's extract the BOLD data for each movie.
  
First we need to just load in the BOLD data.

In [24]:
bolddictionary = dict()
movies = ['MOVIE1_7T_AP', 'MOVIE2_7T_PA', 'MOVIE3_7T_PA', 'MOVIE4_7T_AP']

for i in movies:
    filename = f'/home/jovyan/shared/HCP/115825/MNINonLinear/Results/tfMRI_{i}/tfMRI_{i}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
    cii = ny.load(filename)
    (lh_bold, rh_bold, subcortex_bold) = ny.hcp.cifti_split(cii)
    
    bolddictionary[f'{i}'] = {'lh_bold':lh_bold, 'rh_bold':rh_bold, 'subcortex_bold':subcortex_bold}

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Now that we have the bold dictionary loaded in for each movie, we need to extract only the TRs for when the train or test movies were playing.  
(Keep in mind the following script uses the feature.json used for feature extraction!)

In [25]:
json_filepath = '/home/jovyan/visual-feature-decoding/extract_features/feature.json'

with open(json_filepath) as f:
    data = json.load(f)

# load in json data
for stimuli, stimuli_data in data.items():
        movies = stimuli_data['movies']
        savepath = stimuli_data['savepath']
        TRs = stimuli_data['TRs']
        
        y_train_lh = []
        y_train_rh = []
        y_train_subcortex = []
        y_test_lh = []
        y_test_rh = []
        y_test_subcortex = []
        
        for movie, run_TRs in TRs.items():
            movie_feature = movie[3:-7]
            for keys, values in bolddictionary.items():
                for keyx, region in values.items():
                    movie_bold = keys[:-6]
                    if movie_feature == movie_bold:
                        for key, run in run_TRs.items():
                            if key.startswith('train'):
                                if keyx == 'lh_bold':
                                    y_train_lh.append(region[run[0]:run[1],:])
                                if keyx == 'rh_bold':
                                    y_train_rh.append(region[run[0]:run[1],:])
                                if keyx == 'subcortex_bold':
                                    y_train_subcortex.append(region[run[0]:run[1],:])
                            if key.startswith('test'):
                                if keyx == 'lh_bold':
                                    y_test_lh.append(region[run[0]:run[1],:])
                                if keyx == 'rh_bold':
                                    y_test_rh.append(region[run[0]:run[1],:])
                                if keyx == 'subcortex_bold':
                                    y_test_subcortex.append(region[run[0]:run[1],:])


Now we're moving them from a list to a concatenated numpy array/matrix. I'm sure this could be done in one step, but I did it in two.

In [26]:
# training data concatenation
Y_train_regions = dict()
trainlist = {'y_train_rh': y_train_rh, 'y_train_lh': y_train_lh, 'y_train_subcortex': y_train_subcortex}

for name, train in trainlist.items():
    intermediate = np.concatenate(train, axis=0)
    Y_train_regions[f'{name}'] = intermediate

# test data concatenation
Y_test_regions = dict()
testlist = {'y_test_rh': y_test_rh, 'y_test_lh': y_test_lh, 'y_test_subcortex': y_test_subcortex}

for name, test in testlist.items():
    intermediate = np.stack(test)
    Y_test_regions[f'{name}'] = intermediate

Now we can insert features if we have them! Load them in!

In [40]:
X_train = np.load("/home/jovyan/workingdirectory/me_features_all.npz", allow_pickle=True)['x_train']

They're saved out in features x TRs format

In [41]:
X_train.shape

(1580, 2885)

Now let's mask our BOLD with the visual cortex mask!

In [78]:
rhemi = Y_train_regions['y_train_rh'][:,visualcortexmask['rh']]
lhemi = Y_train_regions['y_train_lh'][:,visualcortexmask['lh']]

Convert all data to the pandas dataframe requested by the classifier team.

In [79]:
# Create a list of arrays where each row is a visual-cortex-sized vector
array_list = [row for row in rhemi]
num_TRs = rhemi.shape[0]

# Create a DataFrame with a single column for the array vectors
rh_df = pd.DataFrame({'BOLD': array_list})
rh_df['subject'] = 115825
rh_df['hemisphere'] = 'RH'
rh_df['TR'] = range(1, num_TRs+1)
featurelist = [row for row in X_train.T]
rh_df['motionenergy'] = featurelist

array_list = [row for row in lhemi]

# Create a DataFrame with a single column for the array vectors
lh_df = pd.DataFrame({'BOLD': array_list})
lh_df['subject'] = 115825
lh_df['hemisphere'] = 'LH'
lh_df['TR'] = range(1, num_TRs+1)
featurelist = [row for row in X_train.T]
lh_df['motionenergy'] = featurelist

combined_df = pd.concat([rh_df, lh_df], ignore_index=True)
new_order = ['subject', 'TR', 'BOLD', 'hemisphere', 'motionenergy']
visualcortex_bold = combined_df[new_order]

In [81]:
visualcortex_bold

,subject,TR,BOLD,hemisphere,motionenergy
0,115825,1,"[10239.255859375, 11871.8603515625, 11993.7880...",RH,"[4.938675064525029, 6.132030972822991, 3.48840..."
1,115825,2,"[10003.9130859375, 12004.6943359375, 11882.559...",RH,"[5.561072885210463, 6.815657111866309, 4.02723..."
2,115825,3,"[10490.705078125, 11965.4453125, 11899.5048828...",RH,"[5.613112008355411, 6.868877894818759, 4.11684..."
3,115825,4,"[10201.9267578125, 12261.61328125, 12231.50097...",RH,"[5.613940827450448, 6.869078405301136, 4.11771..."
4,115825,5,"[10609.5888671875, 11908.8857421875, 12012.480...",RH,"[5.614502650536449, 6.868823924129038, 4.11847..."
...,...,...,...,...,...
5765,115825,2881,"[10969.7568359375, 12294.6904296875, 12354.027...",LH,"[5.125499386899641, 6.301422471960991, 3.64851..."
5766,115825,2882,"[11257.3076171875, 12597.392578125, 12592.3007...",LH,"[5.308308159740405, 6.335240703080079, 3.79914..."
5767,115825,2883,"[11080.234375, 12805.1298828125, 12594.9638671...",LH,"[5.334164146556728, 6.332840959096909, 3.84412..."
5768,115825,2884,"[11247.6865234375, 12280.365234375, 12220.8710...",LH,"[5.300586182654747, 6.323548374880514, 3.83521..."


In [34]:
visualcortex_bold.to_csv('/home/jovyan/workingdirectory/visualcortex_bold.csv', index=False)